In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from pathlib import Path
import os

In [59]:
df = pd.read_csv("dataset.csv")
df.head()

,SMILES,Water Solubility,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,c1(ccccc1)[C@@H](C)Cl,2.178977,5.803148,0.130741,5.803148,0.130741,0.526469,140.613,131.541,140.039278,...,0,0,0,0,0,0,0,0,0,0
1,c1cc(c(cc1N=C=O)Cl)Cl,1.385606,9.802754,0.377932,9.802754,0.377932,0.491538,188.013,184.989,186.959169,...,0,0,0,0,0,0,0,0,0,0
2,C(c1cc([N+]([O-])=O)c(O)cc1)(F)(F)F,2.606381,12.058762,-4.654306,12.058762,0.287037,0.567231,207.107,203.075,207.014328,...,0,0,0,0,0,0,0,0,0,0
3,c1cc(Cl)ccc1NC(=O)OC,2.535294,10.699816,-0.488590,10.699816,0.488590,0.730313,185.610,177.546,185.024356,...,0,0,0,0,0,0,0,0,0,0
4,c1(c(c(c(Cl)c(c1Cl)Cl)Cl)OC)OC,0.201397,5.874491,0.149059,5.874491,0.149059,0.591254,275.946,269.898,273.912190,...,0,0,0,0,0,0,0,0,0,0


In [55]:
def countDoubleBond(df):
    df["num_double_bond"] = df["SMILES"].apply(lambda x : x.count("="))
    
    return df

def countTripleBond(df):
    df["num_triple_bond"] = df["SMILES"].apply(lambda x : x.count("#"))
    
    return df

def countSmallc(df):
    df["num_small_c"] = df["SMILES"].apply(lambda x : x.count("c"))
    
    return df

def countLargeC(df):
    df["num_large_c"] = df["SMILES"].apply(lambda x : x.count("C"))
    return df

In [56]:
df = countDoubleBond(df)
df = countTripleBond(df)
df = countSmallc(df)
df = countLargeC(df)

In [57]:
df_sort = df.sort_values(by="Water Solubility")

In [167]:
df["SMILES"][0].count("C")

3

In [ ]:
sns.pairplot(data=df)

# Top 20

In [168]:
df_sort.tail(20)

,SMILES,Water Solubility,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,num_double_bond,num_triple_bond,num_small_c,num_large_c
544,c1(ccc(C)cc1)C#N,2.964260,8.378485,0.722778,8.378485,0.722778,0.508502,117.151,110.095,117.057849,...,0,0,0,0,0,0,0,1,6,2
51,OC(=O)c1c(Cl)cccc1F,2.965202,12.635926,-1.353611,12.635926,0.085648,0.708686,174.558,170.526,173.988385,...,0,0,0,0,0,0,1,0,6,2
436,Nc1cc(ccc1O)[N+](=O)[O-],2.966142,10.132816,-0.573611,10.132816,0.013148,0.271725,154.125,148.077,154.037842,...,0,0,0,0,0,0,1,0,6,0
405,c1(ccccc1)SC(C)=O,2.966142,10.576115,0.130741,10.576115,0.130741,0.574347,152.218,144.154,152.029586,...,0,0,0,0,0,0,1,0,6,2
961,c1(ccccc1)NC(C)C,2.968950,3.304722,0.516111,3.304722,0.516111,0.656622,135.210,122.106,135.104799,...,0,0,0,0,0,0,0,0,6,3
918,c1(cc(c(N)cc1)Cl)[N+](=O)[O-],2.969882,10.170545,-0.522500,10.170545,0.051667,0.399215,172.571,167.531,172.003955,...,0,0,0,0,0,0,1,0,6,1
904,c1(c(ccc(c1)Cl)N)C,2.979548,5.661389,0.737685,5.661389,0.737685,0.551296,141.601,133.537,141.034527,...,0,0,0,0,0,0,0,0,6,2
270,c1(c(cccc1Cl)N)C,2.979548,5.733951,0.731481,5.733951,0.731481,0.551296,141.601,133.537,141.034527,...,0,0,0,0,0,0,0,0,6,2
730,c1(ccc(cc1)Cl)OCC(=O)O,2.980912,10.088497,-0.995417,10.088497,0.331714,0.783250,186.594,179.538,186.008372,...,0,0,0,0,0,0,1,0,6,3
578,c1(ccc(O)cc1)[C@@H](CC)C,2.982271,9.011155,0.344494,9.011155,0.344494,0.686860,150.221,136.109,150.104465,...,0,0,0,0,0,0,0,0,6,4


# Bottom 20

In [169]:
df_sort.head(20)

,SMILES,Water Solubility,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,num_double_bond,num_triple_bond,num_small_c,num_large_c
232,c12c3c4c5cccc4ccc3ccc2c2ccccc2c1c5,-3.721246,2.374444,1.337645,2.374444,1.337645,0.288335,276.338,264.242,276.093900,...,0,0,0,0,0,0,0,0,22,0
627,c12c3c4c5cccc4ccc3ccc2ccc2c1c5ccc2,-3.585027,2.269167,1.337043,2.269167,1.337043,0.224530,276.338,264.242,276.093900,...,0,0,0,0,0,0,0,0,22,0
645,c12c(\N=N\c3c(cccc3)OC)c(ccc1cccc2)O,-3.481486,10.028047,0.104026,10.028047,0.104026,0.687545,278.311,264.199,278.105528,...,0,0,0,0,0,0,1,0,16,1
491,c12c3c4c5c(ccc4)cccc5c1cccc2ccc3,-3.397940,2.246574,1.325469,2.246574,1.325469,0.243454,252.316,240.220,252.093900,...,0,0,0,0,0,0,0,0,20,0
154,c12c3c4cc5ccccc5cc4c1cccc2ccc3,-3.096910,2.332778,1.315796,2.332778,1.315796,0.334328,252.316,240.220,252.093900,...,0,0,0,0,0,0,0,0,20,0
503,c1(ccccc1)CCCCCCCCCCCC,-2.991400,2.284461,1.261250,2.284461,1.261250,0.410709,246.438,216.198,246.234751,...,0,0,0,0,8,0,0,0,6,12
747,c12c3c(ccc1cc1ccc4c(c1c2)cccc4)cccc3,-2.987163,2.356481,1.300543,2.356481,1.300543,0.230197,278.354,264.242,278.109550,...,0,0,0,0,0,0,0,0,22,0
614,c12c3c(c4c(cccc4)cc3)ccc2ccc2c1cccc2,-2.966576,2.270278,1.300543,2.270278,1.300543,0.291526,278.354,264.242,278.109550,...,0,0,0,0,0,0,0,0,22,0
1022,c12c(cc3cc4ccccc4cc3c1)cc1ccccc1c2,-2.966576,2.309074,1.301055,2.309074,1.301055,0.291526,278.354,264.242,278.109550,...,0,0,0,0,0,0,0,0,22,0
249,c12c3c4c5cccc4ccc3cc3c1c(ccc2c5)ccc3,-2.896196,2.343241,1.336050,2.343241,1.336050,0.224530,276.338,264.242,276.093900,...,0,0,0,0,0,0,0,0,22,0


# Prepare data for prediction.

In [90]:
columns = []
for column in df.columns:
    if column in ["SMILES", "Water Solubility", "MOL"]:
        continue
  
    columns.append(column)

X = df.loc[:, columns].values
Y = df.loc[:, "Water Solubility"].values

print("X shape : ", X.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

X shape :  (1053, 200)


# Predict water solubility by lightgbm with kfold, k=7

In [91]:
r2_scores = []
K = 7

print("K = ", K)
kf = KFold(n_splits = K, shuffle=True, random_state=42).split(X_train, Y_train)
gbm = [None] * K

for i, (train_idx, test_idx) in enumerate(kf):
    x_train = X_train[train_idx]
    y_train = Y_train[train_idx]
    x_test = X_train[test_idx]
    y_test = Y_train[test_idx]

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_test = lgb.Dataset(x_test, y_test)



    # LightGBM parameters
    params = {
            'task' : 'train',
            'boosting_type' : 'gbdt',
            'objective' : 'regression',
            'metric' : "mse",
            'num_leaves' : 32,
            'learning_rate' : 0.01,
            #'feature_fraction' : 0.9,
            #'bagging_fraction' : 0.8,
            #'bagging_freq': 5,
            'random_state' : 42


    }

    # train
    gbm[i] = lgb.train(
        params,
        lgb_train,
        num_boost_round=10000,
        valid_sets=lgb_test,
        early_stopping_rounds=100, 
        verbose_eval=10000
    )


y_preds = np.array([0.] * len(Y_test))
for x in range(K):
    pred = gbm[x].predict(X_test, num_iteration=gbm[x].best_iteration)
    y_preds += pred


y_preds = (y_preds / K)

r2 = r2_score(y_preds, Y_test)
print("R2 score :", r2)




K =  7
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[532]	valid_0's l2: 0.351712
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1038]	valid_0's l2: 0.335569
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[872]	valid_0's l2: 0.298659
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[504]	valid_0's l2: 0.619619
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[467]	valid_0's l2: 0.516332
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[783]	valid_0's l2: 0.42085
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[998]	valid_0's l2: 0.285394
R2 score : 0.5729984441207295


# R2 Score : 0.714

# Feature importance

In [151]:
importances = []
deletes = []
for column, i in zip(columns, gbm[0].feature_importance()):
    importances.append((column, i))
    if i == 0:
        deletes.append(column)

# Chart the chemical structure from SMILES

In [6]:
def chartChemicalStructure(df):
    paths = []
    dir_name = Path("image")
    os.makedirs(dir_name, exist_ok=True)
    
    for i, smiles in enumerate(df["SMILES"]):
        path = dir_name / (str(i).zfill(4) + ".png")
        mol = Chem.MolFromSmiles(smiles)
        Draw.MolToFile(mol, path)
        

# Output descriptor by mordred.

In [2]:
from rdkit import Chem
from mordred import Calculator, descriptors
import pandas as pd

In [11]:
# 計算したい分子のSMILES式のデータフレームを作成

# SMILESからMolオブジェクトに変換
df['MOL'] = df['SMILES'].apply(Chem.MolFromSmiles)

# 記述子を計算
calc = Calculator(descriptors, ignore_3D=True)
df_mordred = calc.pandas(df['MOL'])

df_descriptors = df_mordred.astype(str)
masks = df_descriptors.apply(lambda d: d.str.contains('[a-zA-Z]' ,na=False))
df_descriptors = df_descriptors[~masks]
df_mordred = df_descriptors.astype(float)

df_mordred.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,6.542301,6.236096,0.0,0.0,11.189957,2.193993,4.387987,11.189957,1.243329,3.089765,...,8.590258,37.289972,140.039278,7.779960,88.0,9.0,40.0,43.0,3.472222,2.111111
1,7.956514,7.490313,0.0,0.0,13.419575,2.268353,4.536707,13.419575,1.219961,3.286719,...,8.945463,40.621454,186.959169,13.354226,160.0,13.0,50.0,55.0,4.583333,2.611111
2,10.561930,9.990291,0.0,0.0,15.750620,2.379209,4.758419,15.750620,1.125044,3.540005,...,9.542446,45.789283,207.014328,11.500796,289.0,20.0,70.0,79.0,7.256944,2.972222
3,8.704061,8.015136,0.0,0.0,14.920255,2.227131,4.454263,14.920255,1.243355,3.368115,...,8.874168,41.736132,185.024356,9.251218,216.0,13.0,54.0,58.0,4.833333,2.833333
4,10.094413,10.168570,0.0,0.0,17.189372,2.438931,4.877863,17.189372,1.227812,3.535020,...,9.660078,45.905037,273.912190,13.695610,271.0,25.0,68.0,82.0,7.166667,3.333333


In [14]:
df_concat = pd.concat([df, df_mordred], axis=1)

In [96]:
import deepchem

AttributeError: partially initialized module 'deepchem' has no attribute 'utils' (most likely due to a circular import)